In [1]:
import pandas as pd
import numpy as np
from google.colab import files

In [2]:
!pip install tensorflow==2.5

In [3]:
!pip install keras==2.4.3

  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0


In [4]:
import h5py as h5
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from tensorflow.keras.preprocessing import sequence
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten, Embedding, Dropout, Activation, Reshape
#from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, GlobalAveragePooling1D, TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [5]:
#dataset = pd.read_csv('sample_data/data_CKD_MI.csv')

In [19]:
dataset = pd.read_csv('sample_data/data_CKD_bigdata_refined.csv')


In [20]:
dataset.head()

,sex,age,sido,height,weight,waist,vision_left,vision_right,listen_left,listen_right,BP_HIGH,BP_LWST,BLDS,TOT_CHOLE,TRIGLYCERIDE,HDL_CHOLE,LDL_CHOLE,HMG,OLIG_PROTE_CD,CREATININE,SGOT_AST,SGPT_ALT,GAMMA_GTP,SMK_STAT_TYPE_RSPS_CD,DRINK_OR_NOT,MOUTH_CHECK,BMI,GFR,stage
0,1.0,8.0,43.0,170.0,75.0,90.0,1.0,1.0,1.0,1.0,120.0,80.0,99.0,193.0,92.0,48.0,126.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,1.0,1.0,25.951557,86.886038,2
1,1.0,7.0,11.0,180.0,80.0,89.0,0.9,1.2,1.0,1.0,130.0,82.0,106.0,228.0,121.0,55.0,148.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,0.0,1.0,24.691358,100.644197,1
2,1.0,9.0,41.0,165.0,75.0,91.0,1.2,1.5,1.0,1.0,120.0,70.0,98.0,136.0,104.0,41.0,74.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,0.0,0.0,27.548209,95.928640,1
3,1.0,11.0,48.0,175.0,80.0,91.0,1.5,1.2,1.0,1.0,145.0,87.0,95.0,201.0,106.0,76.0,104.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,0.0,1.0,26.122449,73.203730,2
4,1.0,11.0,30.0,165.0,60.0,80.0,1.0,1.2,1.0,1.0,138.0,82.0,101.0,199.0,104.0,61.0,117.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,0.0,1.0,22.038567,105.714492,1


In [21]:
dataset.shape

(1000000, 29)

In [22]:
dataset.isnull().sum()

sex                      0
age                      0
sido                     0
height                   0
weight                   0
waist                    0
vision_left              0
vision_right             0
listen_left              0
listen_right             0
BP_HIGH                  0
BP_LWST                  0
BLDS                     0
TOT_CHOLE                0
TRIGLYCERIDE             0
HDL_CHOLE                0
LDL_CHOLE                0
HMG                      0
OLIG_PROTE_CD            0
CREATININE               0
SGOT_AST                 0
SGPT_ALT                 0
GAMMA_GTP                0
SMK_STAT_TYPE_RSPS_CD    0
DRINK_OR_NOT             0
MOUTH_CHECK              0
BMI                      0
GFR                      0
stage                    0
dtype: int64

In [10]:
dataset.iloc[:,:-1]

,sex,age,sido,height,weight,waist,vision_left,vision_right,listen_left,listen_right,BP_HIGH,BP_LWST,BLDS,TOT_CHOLE,TRIGLYCERIDE,HDL_CHOLE,LDL_CHOLE,HMG,OLIG_PROTE_CD,CREATININE,SGOT_AST,SGPT_ALT,GAMMA_GTP,SMK_STAT_TYPE_RSPS_CD,DRINK_OR_NOT,MOUTH_CHECK,BMI,GFR
0,1.0,8.0,43.0,170.0,75.0,90.0,1.0,1.0,1.0,1.0,120.0,80.0,99.0,193.0,92.0,48.0,126.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,1.0,1.0,25.951557,86.886038
1,1.0,7.0,11.0,180.0,80.0,89.0,0.9,1.2,1.0,1.0,130.0,82.0,106.0,228.0,121.0,55.0,148.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,0.0,1.0,24.691358,100.644197
2,1.0,9.0,41.0,165.0,75.0,91.0,1.2,1.5,1.0,1.0,120.0,70.0,98.0,136.0,104.0,41.0,74.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,0.0,0.0,27.548209,95.928640
3,1.0,11.0,48.0,175.0,80.0,91.0,1.5,1.2,1.0,1.0,145.0,87.0,95.0,201.0,106.0,76.0,104.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,0.0,1.0,26.122449,73.203730
4,1.0,11.0,30.0,165.0,60.0,80.0,1.0,1.2,1.0,1.0,138.0,82.0,101.0,199.0,104.0,61.0,117.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,0.0,1.0,22.038567,105.714492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159888,1.0,12.0,26.0,155.0,55.0,86.0,0.7,0.8,1.0,1.0,100.0,70.0,125.0,127.0,93.0,51.0,57.0,13.8,1.0,1.4,21.0,11.0,30.0,3.0,0.0,1.0,22.892820,54.490093
159889,1.0,12.0,47.0,165.0,65.0,82.9,0.7,1.0,1.0,1.0,120.0,75.0,92.0,232.0,120.0,64.0,144.0,15.3,2.0,0.9,21.0,27.0,36.0,2.0,1.0,0.0,23.875115,90.730529
159890,1.0,6.0,11.0,170.0,80.0,94.0,1.0,1.0,1.0,1.0,110.0,70.0,84.0,232.0,51.0,86.0,136.0,14.6,1.0,1.0,41.0,32.0,43.0,3.0,1.0,1.0,27.681661,91.748848
159891,2.0,9.0,46.0,170.0,55.0,70.0,1.2,1.5,1.0,1.0,139.0,83.0,120.0,209.0,164.0,55.0,121.0,10.9,1.0,0.7,30.0,32.0,14.0,1.0,0.0,0.0,19.031142,95.127243


In [23]:
dataset.iloc[:,-1]

0         2
1         1
2         1
3         2
4         1
         ..
999995    2
999996    2
999997    2
999998    1
999999    1
Name: stage, Length: 1000000, dtype: int64

In [24]:
# Data preprocessing
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [25]:
from sklearn.model_selection import train_test_split    
from sklearn.preprocessing import MinMaxScaler

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 2)

In [15]:
scaler = MinMaxScaler()

In [16]:
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [29]:
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import keras.layers

In [30]:
from  keras.utils import to_categorical

y_train_new = to_categorical(y_train,dtype="uint8")
print(y_train_new.shape)
y_test = to_categorical(y_test,dtype ="uint8")
print(y_test.shape)
# (unique,counts)=np.unique(y_train,return_counts=True)
# print(unique)
# print(counts)

(800000, 6)
(200000, 6)


In [32]:
def evaluate_model(X_train, X_val, y_train, y_val):
  model = LSTM
  timesteps=125
  dims=4
  model=Sequential()
  model.add(keras.layers.Dense (15,input_shape=(X_train.shape[1],),activation='relu')) #input layer
  model.add(keras.layers.Dense (10,activation='relu')) #hidden
  model.add(keras.layers.Dense (6,activation='softmax')) #output
  model.compile(
  optimizer = 'adam',
  loss = 'binary_crossentropy',
  metrics = ['accuracy']
   )
  # model.add(LSTM(6, return_sequences=True,
  # activation='tanh', recurrent_activation='tanh',
  # dropout=0.2,recurrent_dropout=0.2))
  # model.add(LSTM(25, return_sequences=True,
  # activation='tanh', recurrent_activation='tanh',
  # dropout=0.2,recurrent_dropout=0.2))
  # model.add(LSTM(units=25))
  # model.add(Dense(6,activation='softmax'))
  epochs = 400
  batch_size = 64
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=100, batch_size=128, verbose=2)
  val_acc = model.evaluate(X_val, y_val, verbose = 1)
  return model, val_acc



In [ ]:
#evaluate_model()

In [33]:
n_folds = 1
cv_scores, model_history = list(), list()
for _ in range(n_folds):  
  # split data
  # x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state = np.random.randint(1,1000, 1)[0])
  # evaluate model
  print(X_train.shape)
  print(y_train.shape)
  print(X_test.shape)
  print(y_test.shape)
  model = tf.keras.Sequential()
  model, test_acc = evaluate_model(X_train, X_test, y_train, y_test)
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])

(800000, 28)
(800000,)
(200000, 28)
(200000, 6)
Epoch 1/100


ValueError: ignored